In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
import argparse
import functools
import hashlib
import itertools
import json
import os
import sys
from copy import deepcopy
from functools import partial, reduce
from glob import glob
from multiprocessing import Pool
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import portalocker as pl
import seaborn as sns
from matplotlib import cm
from tqdm import tqdm
from matplotlib.lines import Line2D
from copy import deepcopy
from plot_metrics import preprocess_metrics
from matplotlib.ticker import LogLocator
import matplotlib.patches as mpatches
from util.metrics_utils import *
from util.colour_mapper import colour_mapper
import warnings
warnings.filterwarnings('ignore')

In [256]:
def clean_results(_df_tr, df_tr, _df_ti):    
    appr_list = ['joint', 'backbonefreezing' ]
    t = _df_tr[~((_df_tr['Approach'].isin(appr_list)) & (_df_tr['Memory Size'] == 0))]
    _df_tr = deepcopy(t)

    t = df_tr[~((df_tr['Approach'].isin(appr_list)) & (df_tr['Memory Size'] == 0))]
    df_tr = deepcopy(t)    
    
    t = _df_ti[~((_df_ti['Approach'].isin(appr_list)) & (_df_ti['Memory Size'] == 0))]
    _df_ti = deepcopy(t) 

    t = df_tr[~((df_tr['Approach']=='scratch') & (df_tr['Memory Size'] !=0))]
    df_tr = deepcopy(t)

    t = df_tr[~((df_tr['Approach']=='bic') & (df_tr['Memory Size'] != 1000))]
    df_tr = deepcopy(t)
    
    return _df_tr, df_tr, _df_ti


#correzione eeil train
def eeil_correction(_df_ti, group):
    _df_tr_eeil = _df_tr[(_df_tr['Approach']=='eeil') & (_df_tr['Task']==1)]
    temp = _df_tr_eeil.groupby(group)['Epoch'].max().reset_index()
    temp_time = _df_tr_eeil.groupby([group])[[group,'Train Time']].head(3).reset_index()
    temp_time = temp_time.groupby(group).mean().reset_index()
    max_epochs_eeil = max_epochs[max_epochs['Approach']=='eeil']
    temp = pd.merge(max_epochs_eeil, temp_time, on=[group])
    temp['Time'] = temp[['Epoch', 'Train Time_y']].apply(lambda x: x[0] * x[1], axis=1)
    for c in temp[group].unique():
        _df_ti.loc[_df_ti[(_df_ti['Approach']=='eeil') & (_df_ti['Type']=='train') & (_df_ti['Task']==1) & (_df_ti[group]==c)].index.values[0], 'Time'] = temp[temp[group]==c]['Time'].values[0]

    return _df_ti

#correzione eeil train mincpu
def eeil_correction_mincpu(_df_ti, group):
    _df_tr_eeil = _df_tr[(_df_tr['Approach']=='eeil') & (_df_tr['Task']==1)]
    temp = _df_tr_eeil.groupby(group)['Epoch'].max().reset_index()
    temp_time = _df_tr_eeil.groupby([group])[[group,'Train Time']].head(3).reset_index()
    temp_time = temp_time.groupby(group).min().reset_index()
    max_epochs_eeil = max_epochs[max_epochs['Approach']=='eeil']
    temp = pd.merge(max_epochs_eeil, temp_time, on=[group])
    temp['Time'] = temp[['Epoch', 'Train Time_y']].apply(lambda x: x[0] * x[1], axis=1)
    for c in temp[group].unique():
        _df_ti.loc[_df_ti[(_df_ti['Approach']=='eeil') & (_df_ti['Type']=='train') & (_df_ti['Task']==1) & (_df_ti[group]==c)].index.values[0], 'Time'] = temp[temp[group]==c]['Time'].values[0]

    return _df_ti

#correzione eeil train mingpu
def eeil_correction_mingpu(_df_ti, group):
    _df_tr_eeil = df_tr[(df_tr['Approach']=='eeil') & (df_tr['Task']==1)]
    temp = _df_tr_eeil.groupby(group)['Epoch'].max().reset_index()
    temp_time = _df_tr_eeil.drop(_df_tr_eeil.groupby([group]).tail(40).index, axis=0)
    temp_time = temp_time.groupby(group).min().reset_index()
    max_epochs_eeil = max_epochs[max_epochs['Approach']=='eeil']
    temp = pd.merge(max_epochs_eeil, temp_time, on=[group])
    temp['Time'] = temp[['Epoch_x', 'Train Time_y']].apply(lambda x: x[0] * x[1], axis=1)
    for c in temp[group].unique():
        _df_ti.loc[_df_ti[(_df_ti['Approach']=='eeil') & (_df_ti['Type']=='train') & (_df_ti['Task']==1) & (_df_ti[group]==c)].index.values[0], 'Time'] = temp[temp[group]==c]['Time'].values[0]
    return _df_ti

In [279]:
def divided_barplot_settings(ax1,ax2,ymax):
    d = .015  
    kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
    ax1.plot((-d, +d), (-d, +d), **kwargs)        
    ax1.plot((1 - d, 1 + d), (-d, +d), **kwargs)  
    kwargs.update(transform=ax2.transAxes)  
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs) 
    ax1.set_xlabel('')
    ax2.set_xlabel('')
    ax1.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax1.set(ylabel=None)
    ax2.set(ylabel=None)  
    ax2.xaxis.tick_bottom() 
    ax1.tick_params(bottom=False)  # don't put tick labels at the top
    ax2.tick_params(labeltop=False)  # don't put tick labels at the top

def add_vertical_lines(ax, ymax, ord_list):
    for i,b in enumerate(ax.patches):
        if ord_list[i]=='_': 
            x,y = b.get_xy()
            plt.vlines(x = x+b.get_width()/2, ymin = 0, ymax = ymax, color = 'gray', 
                       linewidth=1.5, linestyle='dashed')
            
def add_vertical_lines_divided(ax1,ax2,ymax,ord_list):
    for i,b in enumerate(ax1.patches):
        if ord_list[i]=='_':
            x,y = b.get_xy()
            x2,y2 = ax2.patches[i].get_xy()
            ax1.vlines(x = x+b.get_width()/2, ymin = 0, ymax = ymax, color = 'gray', 
                           linewidth=1.5, linestyle='dashed')
            ax2.vlines(x = x2+b.get_width()/2, ymin = 0, ymax = ymax, color = 'gray', 
                           linewidth=1.5, linestyle='dashed')

        
def add_vertical_lines_hatches_colors_divided(ax1,ax2,ymax,ord_list,palette):
    '''
    for i,b in enumerate(ax1.patches[:14]):
        if ord_list[i]=='_': 
            x,y = b.get_xy()
            x2,y2 = ax2.patches[i].get_xy()
            ax1.vlines(x = x+b.get_width()/2, ymin = 0, ymax = ymax, color = 'gray', linewidth=1.5)
            ax2.vlines(x = x2+b.get_width()/2, ymin = 0, ymax = ymax, color = 'gray', linewidth=1.5)
    '''
    for i,bar in enumerate (ax1.patches):
        bar.set_facecolor(palette[ord_list[i%12]])
        ax2.patches[i].set_facecolor(palette[ord_list[i%12]])
        if i>=24:
            bar.set(hatch = '////')
            ax2.patches[i].set(hatch = '////')            

def add_vertical_lines_hatches_colors(ax, ymax,ord_list, palette):
    for i,b in enumerate(ax.patches[:len(ord_list)]):
        if ord_list[i]=='_': 
            x,y = b.get_xy()
            ax.vlines(x = x+b.get_width()/2, ymin = 0, ymax = ymax, color = 'gray', 
                    linewidth=1.5)
    for i,bar in enumerate (ax.patches):                
        bar.set_facecolor(palette[ord_list[i%12]])
        if i>=24:
            bar.set(hatch = '////')
            
def plot_train_time(df_ti, type, appr_task, group, nclasses, normalize, figsize=None):
    ord_list = ['lwfgkd','_','jointmem', 'backbonefreezingmem', 'icarlp', 'lucir', 'ssil', '_','bic', 'eeil', 'ewc','icarl', 'il2m','chen2021']
    ord_list = ['lwfgkd','jointmem', 'backbonefreezingmem', 'icarlp', 'lucir', 'ssil','bic', 'eeil', 'ewc','icarl', 'il2m','chen2021']
    ord_list = ['bic', 'eeil', 'ewc', 'jointmem', 'backbonefreezingmem', 'icarl', 'icarlp', 'il2m', 'lucir','lwfgkd', 'chen2021', 'ssil']
    palette = dict([(k, colour_mapper()[k]) for k in ord_list])
    labels = [appr_dict[item] if item!='_' else ' ' for item in ord_list]
    labels = [appr_dict[item] if item!='lwfgkd' else 'LWF' for item in ord_list]
    if type=='all':
        sns.set_style("whitegrid")
        df = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']!='test')].groupby(['Approach',group])['Time'].sum().reset_index()
        if normalize: df['Time'] = df['Time'].div(nclasses)
        f, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True)
        bars1 = sns.barplot(x='Approach', y='Time' , data=df, ax=ax1, order=ord_list, palette=palette)#, ci='sd')
        bars2 = sns.barplot(x='Approach', y='Time' , data=df, ax=ax2, order=ord_list, palette=palette)#, ci='sd')
        bars1.legend(ax1.patches, [appr_dict[i] for i in ord_list], bbox_to_anchor=(1.05, 1),
                     loc=2, borderaxespad=0.1, ncol=1, fontsize=12, frameon=False)
        ax2.set_ylim(0, 450)
        ax1.set_ylim(1000, 3800)
        if not normalize:
            ax2.set_ylim(0*nclasses, 450*nclasses)  
            ax1.set_ylim(1000*nclasses, 3800*nclasses)
        ax2.set_ylabel('Time [s]')
        ax2.yaxis.set_label_coords(-0.15,1.02)
        ymax = 3800 if normalize else 3800*nclasses
        add_vertical_lines_divided(ax1,ax2,ymax,ord_list)                    
        divided_barplot_settings(ax1,ax2,ymax)
        plt.xticks([])
        sns.reset_defaults()

    elif type=='all_normalized_scratch':
        sns.reset_defaults()
        df = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']!='test')].groupby(['Approach',group])['Time'].sum().reset_index()
        if normalize: df['Time'] = df['Time'].div(nclasses)
        f, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True)
        for s in df[group].unique():
            df.loc[(df[group]==s),'Time'] = df.loc[(df[group]==s),'Time'].div(df[(df[group]==s) & (df['Approach']=='scratch')]['Time'].values[0])
        df = df[~(df['Approach']=='scratch')]
        describe = df.groupby('Approach')['Time'].describe()[['mean','std']].reset_index()
        describe[describe.Approach.isin(ord_list)].to_csv(type+'_'+str(nclasses)+'.csv')
        if nclasses==20:
            bars1 = sns.barplot(x='Approach', y='Time' , data=df, ax=ax1, order=ord_list, palette=palette)#, ci='sd')
            bars2 = sns.barplot(x='Approach', y='Time' , data=df, ax=ax2, order=ord_list, palette=palette)#, ci='sd')            
            ax1.set_ylim(9.5, 11.5)  
            ax2.set_ylim(0, 1.1)  
            ymax = 20
            divided_barplot_settings(ax1,ax2,ymax)
            add_vertical_lines_divided(ax1,ax2,ymax,ord_list)
            ax1.yaxis.grid(True)
            ax1.set_axisbelow(True)
            ax2.yaxis.grid(True)
            ax2.set_axisbelow(True)
            ax2.set_ylabel('Ratio of Scratch Total Training Time')
            ax2.yaxis.set_label_coords(-0.10,1.02)                                             
            ax2.set_xticklabels(labels)
        else:
            plt.close()
            bars1 = sns.barplot(x='Approach', y='Time' , data=df, ax=ax, order=ord_list, palette=palette)#, ci='sd')
            ax.set_ylim(0, 0.06)
            ax.set_ylabel('Ratio of Scratch Total Training Time')
            ymax=20
            add_vertical_lines(ax, ymax, ord_list)
            ax.set_xticklabels(labels)
            
        plt.xticks(rotation=45)
        ax.set_xlabel('')
        ax.yaxis.grid(True)
        ax.set_axisbelow(True)
        sns.reset_defaults() 

    elif type=='stacked':
        df = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']!='test') & (df_ti['Approach'].isin(ord_list))]
        df_total = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']!='test')].groupby(['Approach',group])['Time'].sum().reset_index()
        times={}
        t = ['post_train', 'train', 'pre_train']
        for time_type in t:
            df_temp= df[df['Type']==time_type].reset_index()
            for s in df_temp[group].unique():
                df_temp.loc[(df_temp[group]==s),'Time'] = df_temp.loc[(df_temp[group]==s),'Time'].div(df_total[(df_total[group]==s) & (df_total['Approach']=='scratch')]['Time'].values[0])
            df_temp = df_temp[~(df_temp['Approach']=='scratch')]
            df_temp = df_temp.groupby('Approach')['Time'].mean().reset_index()
            #new_row = {'Approach':'_', 'Time':0}
            #df_temp = df_temp.append(new_row, ignore_index=True)
            #df_temp = df_temp.append(new_row, ignore_index=True)
            times[time_type] = df_temp['Time'].values  

        approach = df_temp['Approach']
        index = [appr_dict[i] for i in approach.values]
        ord_list_new = [appr_dict[i] for i in ord_list]
        df = pd.DataFrame({'Approach':index, 'Pre_Train':times['pre_train'], 'Train':times['train'], 'Post_Train':times['post_train'] })
        df = df.set_index('Approach').loc[ord_list_new].reset_index()
        #df = df.drop(labels=[2,9], axis=0).set_index('Approach')
        print(df)
        patch1 = mpatches.Patch(color='gainsboro', label='Post-SGD', ec='black',lw=1,hatch='////')
        patch2 = mpatches.Patch(color='gainsboro', label='SGD', ec='black',lw=1)
        handles = [patch1, patch2]
        
        if nclasses==1:
            ax.set_ylabel('Ratio of Scratch Total Training Time', fontsize=13)
            bars = df.plot.bar(stacked=True, ax=ax, alpha=.80, edgecolor='black',width=0.7, legend=True) 
            ymax=0.06
            ax.set_xticklabels(labels, fontsize=11)
            plt.xticks(rotation=45)   
            plt.xlabel('')
            plt.ylim([0,ymax])
            ax.yaxis.grid(True, linestyle='--')
            ax.set_axisbelow(True)
            ax.legend(handles=handles, facecolor='white', ncol=1, frameon=True, handlelength=1.50, columnspacing=5, fontsize=12, handletextpad=0.5, numpoints=1)
            add_vertical_lines_hatches_colors(ax,ymax,ord_list,palette)
        else:
            f, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True)
            ax2.set_ylim(0, 1.1)  #era 1.1
            ax1.set_ylim(9.5, 10.6) #era 11
            bars1 = df.plot.bar(stacked=True, ax=ax1, alpha=.80, edgecolor='black',width=0.7, legend=True) 
            bars2 = df.plot.bar(stacked=True, ax=ax2, alpha=.80, edgecolor='black',width=0.7, legend=False)                     
            ax1.yaxis.grid(True, linestyle='--')
            ax1.set_axisbelow(True)
            ax2.yaxis.grid(True, linestyle='--')
            ax2.set_axisbelow(True)
            ymax = 20
            divided_barplot_settings(ax1,ax2,ymax)
            plt.xticks(rotation=45)
            ax.set_xlabel('')
            ax.yaxis.grid(True)
            ax.set_axisbelow(True)
            ax2.set_xticklabels(labels, fontsize=11)
            ax2.set_ylabel('Ratio of Scratch Total Training Time', fontsize=13)
            ax2.yaxis.set_label_coords(-0.12,1.02)
            add_vertical_lines_hatches_colors_divided(ax1,ax2,ymax,ord_list,palette)
            ax1.legend(handles=handles, facecolor='white', ncol=1, frameon=True, handlelength=1.50, columnspacing=5, fontsize=12, handletextpad=0.5, numpoints=1)

        
    elif type=='post_train_percentage':
        dft1 = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']!='test')]
        df1 = dft1.groupby(['Approach',group])['Time'].sum().reset_index()
        df1['Time'] = df1['Time'].div(nclasses)
        df_all = deepcopy(df1)
        df_all = df_all.sort_values(['Approach', group]).reset_index()
        sns.reset_defaults()
        df1 = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']=='post_train')]
        df1 = df1.sort_values(['Approach', group]).reset_index()
        df=deepcopy(df1)
        df['Time'] = df['Time'].div(nclasses).round(2)
        df['Time'] = 100*df['Time'].div(df_all['Time'])
        bars = sns.barplot(x='Approach', y='Time' , data=df, ax=ax, order=ord_list, palette=palette)#, ci='sd')
        plt.xticks(rotation=45)
        ax.set_xticklabels(labels)
        plt.ylabel('Percentage of Total Training Time')
        plt.xlabel('')
        axx = plt.gca()
        axx.yaxis.grid(True)
        axx.set_axisbelow(True)
        plt.ylim([0,40])
        ymax=40
        add_vertical_lines(axx,ymax,ord_list)
        
    else:
        sns.reset_defaults()
        df1 = df_ti[(df_ti['Task']==appr_task) & (df_ti['Type']==type)]
        df1 = df1.sort_values(['Approach', group]).reset_index()
        df=deepcopy(df1)
        if normalize:
            df['Time'] = df['Time'].div(nclasses).round(2)
        else:
            df['Time'] = df['Time'].round(2)
        bars = sns.barplot(x='Approach', y='Time' , data=df, ax=ax, order=ord_list, palette=palette)#, ci='sd')
        describe = df.groupby('Approach')['Time'].describe()[['mean','std']].reset_index()
        describe[describe.Approach.isin(ord_list)].to_csv(type+'_'+str(nclasses)+'.csv')
        plt.xticks(rotation=45)
        ax.set_xticklabels(labels)
        plt.ylabel('Time [s]')
        plt.xlabel('')
        axx = plt.gca()
        bars.set_yscale("log")
        axx.yaxis.set_minor_locator(LogLocator(base=10, subs='auto'))
        ymax= 110 if normalize else 110*nclasses
        plt.ylim([0,ymax])
        add_vertical_lines(axx,ymax,ord_list)

In [283]:
#20+20 times
#load old experiments for info about epochs
exp_name = 'base20_incr20_stop2'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_tr, df_ti = get_training_info(df_stdout_filenames)
df_tr = df_tr[df_tr['Seed']!=0]
df_ti = df_ti[df_ti['Seed']!=0]
#load old scratch experiments for info about epochs
exp_name = 'mirage_generic_scratch'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr_scratch, _df_ti_scratch = get_training_info(df_stdout_filenames)
_df_tr_scratch = _df_tr_scratch[_df_tr_scratch['Base Apps'].isin([20, 40])]
_df_ti_scratch = _df_ti_scratch[_df_ti_scratch['Base Apps'].isin([20, 40])]
_df_tr_scratch.loc[_df_tr_scratch['Base Apps'] == 20, 'Task'] = 0
_df_ti_scratch.loc[_df_ti_scratch['Base Apps'] == 20, 'Task'] = 0
_df_tr_scratch.loc[_df_tr_scratch['Base Apps'] == 40, 'Task'] = 1
_df_ti_scratch.loc[_df_ti_scratch['Base Apps'] == 40, 'Task'] = 1
#load new experiments (controlled)
exp_name = 'timing_base20'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr, _df_ti = get_training_info(df_stdout_filenames)

_df_tr = _df_tr[_df_tr['Last App']==-1]
_df_tr = _df_tr[_df_tr['Seed']!=0]
_df_ti = _df_ti[_df_ti['Seed']!=0]

_df_tr, df_tr, _df_ti = clean_results(_df_tr, df_tr, _df_ti)

final_df = pd.concat((df_tr[df_tr['Task'] == 1], _df_tr_scratch[_df_tr_scratch['Task'] == 1]), axis=0)
final_df = final_df[final_df['Retrained App']==-1]

mean_time_per_epoch = _df_tr[_df_tr['Task'] == 1].groupby(['Seed', 'Approach', 'Last App', 'First Momentum',
                        'Base Momentum', 'Network', 'Out Features Size', 'Model App'])['Train Time'].mean().reset_index()

max_epochs = final_df.groupby(['Seed', 'Approach', 'Last App', 'First Momentum', 'Base Momentum', 
                               'Network', 'Out Features Size', 'Model App'])['Epoch'].max().reset_index()


columns = ['Approach', 'Seed', 'Model App']
groups = list(mean_time_per_epoch.groupby(columns).groups)
for group in tqdm(groups):

    _filter = reduce(lambda x, y: x & y, [mean_time_per_epoch[col] == g for col, g in zip(columns, group)])
    train_time = mean_time_per_epoch.loc[_filter]['Train Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, group)])
    if sum(_filter) == 0:
        continue
    
    max_epochs.loc[_filter, 'Train Time'] = train_time
max_epochs['Total Time'] = max_epochs[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)

max_epochs = max_epochs.groupby(['Approach', 'Seed']).sum().reset_index()

groups = list(_df_ti.groupby(['Approach', 'Seed', 'Last App']).groups)

failed_apprs = []
columns = ['Approach', 'Seed'] 
i = 0
for group in tqdm(groups):
    _group = group
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, _group)])
    if sum(_filter) == 0:
        failed_apprs.append(group[0])
        continue
    t = max_epochs.loc[_filter, 'Total Time'].values[0]
    
    _filter = reduce(lambda x, y: x & y, [_df_ti[col] == g for col, g in zip(columns, group)])
    _df_ti.loc[(_filter) & (_df_ti['Type'] == 'train') & (_df_ti['Task'] == 1), 'Time'] = t

    
#correzione chen2021 post_train
df_tr_chen = df_tr[(df_tr['Approach']=='chen2021') & (df_tr['Retrained App']!=-1)]
temp = df_tr_chen.groupby(['Seed', 'Retrained App'])['Epoch'].max().reset_index()
temp_time = mean_time_per_epoch.groupby(['Seed', 'Approach'])['Train Time'].mean().reset_index()
temp_time = temp_time[temp_time['Approach']=='chen2021']
temp = pd.merge(temp, temp_time, on=["Seed"])
temp['Retraining Time'] = temp[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)
temp = pd.merge(temp[['Retraining Time', 'Seed']], _df_ti[(_df_ti['Approach']=='chen2021') &
    (_df_ti['Type']=='post_train') & (_df_ti['Task']==1)],on="Seed")
temp['Time'] = temp[['Retraining Time', 'Time']].apply(lambda x: x[0] + x[1], axis=1)

for c in temp['Seed'].unique():
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') & (_df_ti['Task']==1) & (_df_ti['Seed']==c)].index.values[0], 'Time'] = temp[temp['Seed']==c]['Time'].values[0]
    
#correzione eeil train
_df_ti = eeil_correction(_df_ti, group='Seed')

_df_ti.loc[_df_ti['Approach']=='joint', 'Approach'] = 'jointmem'
_df_ti.loc[_df_ti['Approach']=='backbonefreezing', 'Approach'] = 'backbonefreezingmem'
_df_ti.loc[_df_ti['Approach']=='lwf','Approach'] = 'lwfgkd'

_df_ti_20 = deepcopy(_df_ti)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 650/650 [00:01<00:00, 508.36it/s]


In [215]:
#20+20 times min cpu
#load old experiments for info about epochs
exp_name = 'base20_incr20_stop2'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_tr, df_ti = get_training_info(df_stdout_filenames)
df_tr = df_tr[df_tr['Seed']!=0]
df_ti = df_ti[df_ti['Seed']!=0]
#load old scratch experiments for info about epochs
exp_name = 'mirage_generic_scratch'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr_scratch, _df_ti_scratch = get_training_info(df_stdout_filenames)
_df_tr_scratch = _df_tr_scratch[_df_tr_scratch['Base Apps'].isin([20, 40])]
_df_ti_scratch = _df_ti_scratch[_df_ti_scratch['Base Apps'].isin([20, 40])]
_df_tr_scratch.loc[_df_tr_scratch['Base Apps'] == 20, 'Task'] = 0
_df_ti_scratch.loc[_df_ti_scratch['Base Apps'] == 20, 'Task'] = 0
_df_tr_scratch.loc[_df_tr_scratch['Base Apps'] == 40, 'Task'] = 1
_df_ti_scratch.loc[_df_ti_scratch['Base Apps'] == 40, 'Task'] = 1
#load new experiments (controlled)
exp_name = 'timing_base20'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr, _df_ti = get_training_info(df_stdout_filenames)

_df_tr = _df_tr[_df_tr['Last App']==-1]
_df_tr = _df_tr[_df_tr['Seed']!=0]
_df_ti = _df_ti[_df_ti['Seed']!=0]

_df_tr, df_tr, _df_ti = clean_results(_df_tr, df_tr, _df_ti)

final_df = pd.concat((df_tr[df_tr['Task'] == 1], _df_tr_scratch[_df_tr_scratch['Task'] == 1]), axis=0)
final_df = final_df[final_df['Retrained App']==-1]

mean_time_per_epoch = _df_tr[_df_tr['Task'] == 1].groupby(['Seed', 'Approach', 'Last App', 'First Momentum',
                        'Base Momentum', 'Network', 'Out Features Size', 'Model App'])['Train Time'].min().reset_index()

max_epochs = final_df.groupby(['Seed', 'Approach', 'Last App', 'First Momentum', 'Base Momentum', 
                               'Network', 'Out Features Size', 'Model App'])['Epoch'].max().reset_index()

columns = ['Approach', 'Seed', 'Model App']
groups = list(mean_time_per_epoch.groupby(columns).groups)
for group in tqdm(groups):

    _filter = reduce(lambda x, y: x & y, [mean_time_per_epoch[col] == g for col, g in zip(columns, group)])
    train_time = mean_time_per_epoch.loc[_filter]['Train Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, group)])
    if sum(_filter) == 0:
        continue
    
    max_epochs.loc[_filter, 'Train Time'] = train_time
max_epochs['Total Time'] = max_epochs[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)

max_epochs = max_epochs.groupby(['Approach', 'Seed']).sum().reset_index()

groups = list(_df_ti.groupby(['Approach', 'Seed', 'Last App']).groups)

failed_apprs = []
columns = ['Approach', 'Seed'] 
i = 0
for group in tqdm(groups):
    _group = group
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, _group)])
    if sum(_filter) == 0:
        failed_apprs.append(group[0])
        continue
    t = max_epochs.loc[_filter, 'Total Time'].values[0]
    
    _filter = reduce(lambda x, y: x & y, [_df_ti[col] == g for col, g in zip(columns, group)])
    _df_ti.loc[(_filter) & (_df_ti['Type'] == 'train') & (_df_ti['Task'] == 1), 'Time'] = t

    
#correzione chen2021 post_train
df_tr_chen = df_tr[(df_tr['Approach']=='chen2021') & (df_tr['Retrained App']!=-1)]
temp = df_tr_chen.groupby(['Seed', 'Retrained App'])['Epoch'].max().reset_index()
temp_time = mean_time_per_epoch.groupby(['Seed', 'Approach'])['Train Time'].min().reset_index()
temp_time = temp_time[temp_time['Approach']=='chen2021']
temp = pd.merge(temp, temp_time, on=["Seed"])
temp['Retraining Time'] = temp[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)
temp = pd.merge(temp[['Retraining Time', 'Seed']], _df_ti[(_df_ti['Approach']=='chen2021') &
    (_df_ti['Type']=='post_train') & (_df_ti['Task']==1)],on="Seed")
temp['Time'] = temp[['Retraining Time', 'Time']].apply(lambda x: x[0] + x[1], axis=1)

for c in temp['Seed'].unique():
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') & (_df_ti['Task']==1) & (_df_ti['Seed']==c)].index.values[0], 'Time'] = temp[temp['Seed']==c]['Time'].values[0]
    
#correzione eeil train
_df_ti = eeil_correction_mincpu(_df_ti, group='Seed')

_df_ti.loc[_df_ti['Approach']=='joint', 'Approach'] = 'jointmem'
_df_ti.loc[_df_ti['Approach']=='backbonefreezing', 'Approach'] = 'backbonefreezingmem'
_df_ti.loc[_df_ti['Approach']=='lwf','Approach'] = 'lwfgkd'

_df_ti_20_mincpu = deepcopy(_df_ti)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 650/650 [00:01<00:00, 510.10it/s]


In [257]:
#20+20 times min gpu
#load old experiments for info about epochs
exp_name = 'base20_incr20_stop2'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_tr, df_ti = get_training_info(df_stdout_filenames)
df_tr = df_tr[df_tr['Seed']!=0]
df_ti = df_ti[df_ti['Seed']!=0]
#load old scratch experiments for info about epochs
exp_name = 'mirage_generic_scratch'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr_scratch, _df_ti_scratch = get_training_info(df_stdout_filenames)
_df_tr_scratch = _df_tr_scratch[_df_tr_scratch['Base Apps'].isin([20, 40])]
_df_ti_scratch = _df_ti_scratch[_df_ti_scratch['Base Apps'].isin([20, 40])]
_df_tr_scratch.loc[_df_tr_scratch['Base Apps'] == 20, 'Task'] = 0
_df_ti_scratch.loc[_df_ti_scratch['Base Apps'] == 20, 'Task'] = 0
_df_tr_scratch.loc[_df_tr_scratch['Base Apps'] == 40, 'Task'] = 1
_df_ti_scratch.loc[_df_ti_scratch['Base Apps'] == 40, 'Task'] = 1


exp_name = 'timing_base20'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr, _df_ti = get_training_info(df_stdout_filenames)

_df_tr = _df_tr[_df_tr['Last App']==-1]
_df_tr = _df_tr[_df_tr['Seed']!=0]
_df_ti = _df_ti[_df_ti['Seed']!=0]

       
_df_tr, df_tr, _df_ti = clean_results(_df_tr, df_tr, _df_ti)

t = df_ti[~((df_ti['Approach']=='bic') & (df_ti['Memory Size'] != 1000))]
df_ti = deepcopy(t)


appr_list = ['joint', 'backbonefreezing' ]
t = df_ti[~((df_ti['Approach'].isin(appr_list)) & (df_ti['Memory Size'] == 0))]
df_ti = deepcopy(t)

final_df = pd.concat((df_tr[df_tr['Task'] == 1], _df_tr_scratch[_df_tr_scratch['Task'] == 1]), axis=0)
final_df = final_df[final_df['Retrained App']==-1]

final_gpu_time = pd.concat((df_ti[df_ti['Task'] == 1], _df_ti_scratch[_df_ti_scratch['Task'] == 1]), axis=0)

t = final_gpu_time[~((final_gpu_time['Approach']=='scratch') & (final_gpu_time['Memory Size'] !=0))]
final_gpu_time = deepcopy(t)


mean_time_per_epoch = df_tr[df_tr['Task'] == 1].groupby(['Seed', 'Approach', 'Last App', 'First Momentum',
                        'Base Momentum', 'Network', 'Out Features Size', 'Model App'])['Train Time'].min().reset_index()
mean_time_per_epoch_scratch = _df_tr_scratch[(_df_tr_scratch['Approach'] == 'scratch') & (_df_tr_scratch['Task'] == 1)].groupby(['Seed', 'Approach',                                                                                                                                 'Last App', 'First Momentum',
                        'Base Momentum', 'Network', 'Out Features Size', 'Model App'])['Train Time'].min().reset_index()
mean_time_per_epoch = pd.concat([mean_time_per_epoch_scratch, mean_time_per_epoch])


max_epochs = final_df.groupby(['Seed', 'Approach', 'Last App', 'First Momentum', 'Base Momentum', 
                               'Network', 'Out Features Size', 'Model App'])['Epoch'].max().reset_index()

columns = ['Approach', 'Seed', 'Model App']
groups = list(mean_time_per_epoch.groupby(columns).groups)
for group in tqdm(groups):

    _filter = reduce(lambda x, y: x & y, [mean_time_per_epoch[col] == g for col, g in zip(columns, group)])
    train_time = mean_time_per_epoch.loc[_filter]['Train Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, group)])
    if sum(_filter) == 0:
        continue
    
    max_epochs.loc[_filter, 'Train Time'] = train_time
max_epochs['Total Time'] = max_epochs[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)

max_epochs = max_epochs.groupby(['Approach', 'Seed']).sum().reset_index()

groups = list(final_gpu_time.groupby(['Approach', 'Seed', 'Last App']).groups)

failed_apprs = []
columns = ['Approach', 'Seed'] 
i = 0
for group in tqdm(groups):
    _group = group
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, _group)])
    if sum(_filter) == 0:
        failed_apprs.append(group[0])
        continue
    t = max_epochs.loc[_filter, 'Total Time'].values[0]    
    _filter = reduce(lambda x, y: x & y, [final_gpu_time[col] == g for col, g in zip(columns, group)])
    final_gpu_time.loc[(_filter) & (final_gpu_time['Type'] == 'train') & (final_gpu_time['Task'] == 1), 'Time'] = t

    
#correzione chen2021 post_train
df_tr_chen = df_tr[(df_tr['Approach']=='chen2021') & (df_tr['Retrained App']!=-1)]
temp = df_tr_chen.groupby(['Seed', 'Retrained App'])['Epoch'].max().reset_index()
temp_time = mean_time_per_epoch.groupby(['Seed', 'Approach'])['Train Time'].min().reset_index()
temp_time = temp_time[temp_time['Approach']=='chen2021']
temp = pd.merge(temp, temp_time, on=["Seed"])
temp['Retraining Time'] = temp[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)
temp = pd.merge(temp[['Retraining Time', 'Seed']], _df_ti[(_df_ti['Approach']=='chen2021') &
    (_df_ti['Type']=='post_train') & (_df_ti['Task']==1)],on="Seed")
temp['Time'] = temp[['Retraining Time', 'Time']].apply(lambda x: x[0] + x[1], axis=1)

for c in temp['Seed'].unique():
    final_gpu_time.loc[_df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') & (_df_ti['Task']==1) & (_df_ti['Seed']==c)].index.values[0], 'Time'] = temp[temp['Seed']==c]['Time'].values[0]
    
#correzione eeil train
final_gpu_time = eeil_correction_mingpu(final_gpu_time, group='Seed')

final_gpu_time.loc[final_gpu_time['Approach']=='joint', 'Approach'] = 'jointmem'
final_gpu_time.loc[final_gpu_time['Approach']=='backbonefreezing', 'Approach'] = 'backbonefreezingmem'
final_gpu_time.loc[final_gpu_time['Approach']=='lwf','Approach'] = 'lwfgkd'

_df_ti_20_mingpu = deepcopy(final_gpu_time)
#_df_ti_20_mingpu.loc[_df_ti_20_mingpu.Approach=='scratch','Task'] = 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 650/650 [00:01<00:00, 634.35it/s]


In [287]:
#20+20 plot times
'''
types = ['all', 'post_train','post_train_percentage']
for t in types:
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_20, t, appr_task = 1, group='Seed', nclasses = 20, normalize=True, figsize=figsize)
    t=t+'_normalized'
    plt.savefig('ExecTime_Scenario_20+20_%s.pdf' % t, bbox_inches='tight')
    
    
types = ['stacked','all', 'post_train','post_train_percentage','all_normalized_scratch']
for t in types:
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_20, t, appr_task=1, group='Seed', nclasses=20, normalize=False, figsize=figsize)
    plt.savefig('ExecTime_Scenario_20+20_%s.pdf' % t, bbox_inches='tight')
'''

#20+20 plot times min cpu
types = ['all', 'post_train','post_train_percentage']
for t in types:
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_20_mincpu, t, appr_task = 1, group='Seed', nclasses = 20, normalize=True, figsize=figsize)
    t=t+'_normalized'
    plt.savefig('ExecTime_Scenario_20+20_%s_mincpu.pdf' % t, bbox_inches='tight')
    
    
types = ['stacked','all', 'post_train','post_train_percentage','all_normalized_scratch']
for t in types:
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_20_mincpu, t, appr_task=1, group='Seed', nclasses=20, normalize=False, figsize=figsize)
    plt.savefig('ExecTime_Scenario_20+20_%s_mincpu.pdf' % t, bbox_inches='tight')
    
'''
#20+20 plot times min gpu
types = ['all', 'post_train','post_train_percentage']
for t in types:
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_20_mingpu, t, appr_task = 1, group='Seed', nclasses = 20, normalize=True, figsize=figsize)
    t=t+'_normalized'
    plt.savefig('ExecTime_Scenario_20+20_%s_mingpu.pdf' % t, bbox_inches='tight')
    
    
types = ['stacked','all', 'post_train','post_train_percentage','all_normalized_scratch']
for t in types:
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_20_mingpu, t, appr_task=1, group='Seed', nclasses=20, normalize=False, figsize=figsize)
    plt.savefig('ExecTime_Scenario_20+20_%s_mingpu.pdf' % t, bbox_inches='tight')
'''

   Approach     Pre_Train     Train    Post_Train
0       BiC  4.266354e-05  0.728858  1.018616e-02
1      EEIL  0.000000e+00  0.680181  2.708990e-02
2       EWC  0.000000e+00  0.691616  1.151539e-02
3    FT-Mem  0.000000e+00  0.432946  8.598948e-03
4    FZ-Mem  0.000000e+00  0.266974  8.720079e-03
5     iCaRL  0.000000e+00  0.797528  8.786125e-03
6    iCaRL+  0.000000e+00  0.717483  8.673433e-03
7      IL2M  0.000000e+00  0.480197  1.010831e-02
8     LUCIR  3.264089e-07  0.797476  8.711970e-03
9   LwF-GKD  0.000000e+00  0.700408  8.131872e-07
10  OvA-Ens  0.000000e+00  9.773782  3.545583e-01
11     SSIL  0.000000e+00  0.277570  8.678744e-03


"\n#20+20 plot times min gpu\ntypes = ['all', 'post_train','post_train_percentage']\nfor t in types:\n    figsize = (7, 4)\n    fig, ax = plt.subplots(figsize=figsize)\n    plot_train_time(_df_ti_20_mingpu, t, appr_task = 1, group='Seed', nclasses = 20, normalize=True, figsize=figsize)\n    t=t+'_normalized'\n    plt.savefig('ExecTime_Scenario_20+20_%s_mingpu.pdf' % t, bbox_inches='tight')\n    \n    \ntypes = ['stacked','all', 'post_train','post_train_percentage','all_normalized_scratch']\nfor t in types:\n    figsize = (7, 4)\n    fig, ax = plt.subplots(figsize=figsize)\n    plot_train_time(_df_ti_20_mingpu, t, appr_task=1, group='Seed', nclasses=20, normalize=False, figsize=figsize)\n    plt.savefig('ExecTime_Scenario_20+20_%s_mingpu.pdf' % t, bbox_inches='tight')\n"

In [282]:
#39+1 times
#load old experiments for info about epochs
exp_name = '39_1_base39_incr1_stop2'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_tr, df_ti = get_training_info(df_stdout_filenames)
#load new experiments (isolated)
exp_name= 'timing_base39'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr, _df_ti = get_training_info(df_stdout_filenames)
_df_tr = _df_tr[_df_tr['Seed']==0]
_df_tr = _df_tr[_df_tr['Last App']!=-1]
_df_ti = _df_ti[_df_ti['Last App']!=-1]
#load old scratch experiments for info about epochs
exp_name = 'mirage_generic_scratch_39_ub_base39_incr1_stop1'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr_scratch, _df_ti_scratch = get_training_info(df_stdout_filenames)

_df_tr, df_tr, _df_ti = clean_results(_df_tr, df_tr, _df_ti)

final_df = pd.concat((df_tr[df_tr['Task'] == 1], _df_tr_scratch[_df_tr_scratch['Task'] == 0]), axis=0)
final_df = final_df[final_df['Retrained App']==-1]

cols = ['Seed', 'Approach', 'Last App', 'First Momentum', 'Base Momentum', 'Network', 'Out Features Size']
mean_time_per_epoch = _df_tr[_df_tr['Task'] == 1].groupby(cols)['Train Time'].mean().reset_index()
max_epochs = final_df.groupby(cols)['Epoch'].max().reset_index()

columns = ['Approach', 'Seed', 'Last App']
groups = list(mean_time_per_epoch.groupby(columns).groups)
for group in tqdm(groups):
    _filter = reduce(lambda x, y: x & y, [mean_time_per_epoch[col] == g for col, g in zip(columns, group)])
    train_time = mean_time_per_epoch.loc[_filter]['Train Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, group)])
    if sum(_filter) == 0:
        continue
    max_epochs.loc[_filter, 'Train Time'] = train_time
    
max_epochs['Total Time'] = max_epochs[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)

groups = list(_df_ti.groupby(columns).groups)

failed_apprs = []
i = 0
for group in tqdm(groups):
    _group = group
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, _group)])
    if sum(_filter) == 0:
        failed_apprs.append(group[0])
        continue
    t = max_epochs.loc[_filter, 'Total Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [_df_ti[col] == g for col, g in zip(columns, group)])
    _df_ti.loc[(_filter) & (_df_ti['Type'] == 'train') & (_df_ti['Task'] == 1), 'Time'] = t

#correzione chen2021 retraining time
max_epochs_chen = max_epochs[max_epochs['Approach'] == 'chen2021'][['Epoch', 'Train Time', 'Last App']]
df_tr_chen = df_tr[(df_tr['Approach'] == 'chen2021') & (df_tr['Task'] == 1) & (df_tr['Retrained App'] != -1)]
max_epochs_chen = max_epochs_chen[['Train Time', 'Last App']]
temp = df_tr_chen.groupby(['Last App', 'Retrained App'])['Epoch'].max().reset_index().groupby(['Last App']).sum().reset_index()
temp = pd.merge(temp, max_epochs_chen, on="Last App")
temp['Retraining Time'] = temp[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)
temp = pd.merge(temp[['Retraining Time', 'Last App']], _df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') & (_df_ti['Task']==1)],on="Last App")
temp['Time'] = temp[['Retraining Time', 'Time']].apply(lambda x: x[0] + x[1], axis=1)

for c in temp['Last App'].unique():
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') &
    (_df_ti['Task']==1) & (_df_ti['Last App']==c)].index.values[0],'Time'] = temp[temp['Last App']==c]['Time'].values[0]
    
#correzione eeil train
_df_ti = eeil_correction(_df_ti, group='Last App')

for c in _df_ti[_df_ti['Approach']=='ssil']['Last App'].unique():
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='ssil') & (_df_ti['Type']=='post_train')  &
    (_df_ti['Task']==1) & (_df_ti['Last App']==c)].index.values[0],'Time'] = _df_ti[(_df_ti['Approach']=='ssil') 
    & (_df_ti['Type']=='post_train')  & (_df_ti['Task']==1) & (_df_ti['Last App']==c)]['Time'].values[0]-0.2
    
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='icarlp') & (_df_ti['Type']=='post_train')  &
    (_df_ti['Task']==1) & (_df_ti['Last App']==c)].index.values[0],'Time'] = _df_ti[(_df_ti['Approach']=='icarlp') 
    & (_df_ti['Type']=='post_train')  & (_df_ti['Task']==1) & (_df_ti['Last App']==c)]['Time'].values[0]-0.2

    
_df_ti.loc[_df_ti['Approach']=='joint', 'Approach'] = 'jointmem'
_df_ti.loc[_df_ti['Approach']=='backbonefreezing', 'Approach'] = 'backbonefreezingmem'
_df_ti.loc[_df_ti['Approach']=='lwf','Approach'] = 'lwfgkd'

_df_ti_39 = deepcopy(_df_ti)       

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 670/670 [00:01<00:00, 445.86it/s]


In [281]:
#min cpu * epochs on gpu
#39+1 times
#load old experiments for info about epochs
exp_name = '39_1_base39_incr1_stop2'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_tr, df_ti = get_training_info(df_stdout_filenames)
#load new experiments (isolated)
exp_name= 'timing_base39'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr, _df_ti = get_training_info(df_stdout_filenames)
_df_tr = _df_tr[_df_tr['Seed']==0]
_df_tr = _df_tr[_df_tr['Last App']!=-1]
_df_ti = _df_ti[_df_ti['Last App']!=-1]
#load old scratch experiments for info about epochs
exp_name = 'mirage_generic_scratch_39_ub_base39_incr1_stop1'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr_scratch, _df_ti_scratch = get_training_info(df_stdout_filenames)

_df_tr, df_tr, _df_ti = clean_results(_df_tr, df_tr, _df_ti)

final_df = pd.concat((df_tr[df_tr['Task'] == 1], _df_tr_scratch[_df_tr_scratch['Task'] == 0]), axis=0)
final_df = final_df[final_df['Retrained App']==-1]

cols = ['Seed', 'Approach', 'Last App', 'First Momentum', 'Base Momentum', 'Network', 'Out Features Size']
mean_time_per_epoch = _df_tr[_df_tr['Task'] == 1].groupby(cols)['Train Time'].min().reset_index()
max_epochs = final_df.groupby(cols)['Epoch'].max().reset_index()

#print(max_epochs[max_epochs['Approach']=='bic'][['Approach','Epoch','Last App']])

columns = ['Approach', 'Seed', 'Last App']
groups = list(mean_time_per_epoch.groupby(columns).groups)
for group in tqdm(groups):
    _filter = reduce(lambda x, y: x & y, [mean_time_per_epoch[col] == g for col, g in zip(columns, group)])
    train_time = mean_time_per_epoch.loc[_filter]['Train Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, group)])
    if sum(_filter) == 0:
        continue
    max_epochs.loc[_filter, 'Train Time'] = train_time
    
max_epochs['Total Time'] = max_epochs[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)

groups = list(_df_ti.groupby(columns).groups)

failed_apprs = []
i = 0
for group in tqdm(groups):
    _group = group
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, _group)])
    if sum(_filter) == 0:
        failed_apprs.append(group[0])
        continue
    t = max_epochs.loc[_filter, 'Total Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [_df_ti[col] == g for col, g in zip(columns, group)])
    _df_ti.loc[(_filter) & (_df_ti['Type'] == 'train') & (_df_ti['Task'] == 1), 'Time'] = t

#correzione chen2021 retraining time
max_epochs_chen = max_epochs[max_epochs['Approach'] == 'chen2021'][['Epoch', 'Train Time', 'Last App']]
df_tr_chen = df_tr[(df_tr['Approach'] == 'chen2021') & (df_tr['Task'] == 1) & (df_tr['Retrained App'] != -1)]
max_epochs_chen = max_epochs_chen[['Train Time', 'Last App']]
temp = df_tr_chen.groupby(['Last App', 'Retrained App'])['Epoch'].max().reset_index().groupby(['Last App']).sum().reset_index()
temp = pd.merge(temp, max_epochs_chen, on="Last App")
temp['Retraining Time'] = temp[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)
temp = pd.merge(temp[['Retraining Time', 'Last App']], _df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') & (_df_ti['Task']==1)],on="Last App")
temp['Time'] = temp[['Retraining Time', 'Time']].apply(lambda x: x[0] + x[1], axis=1)

for c in temp['Last App'].unique():
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') &
    (_df_ti['Task']==1) & (_df_ti['Last App']==c)].index.values[0],'Time'] = temp[temp['Last App']==c]['Time'].values[0]
    
#correzione eeil train
_df_ti = eeil_correction_mincpu(_df_ti, group='Last App')

for c in _df_ti[_df_ti['Approach']=='ssil']['Last App'].unique():
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='ssil') & (_df_ti['Type']=='post_train')  &
    (_df_ti['Task']==1) & (_df_ti['Last App']==c)].index.values[0],'Time'] = _df_ti[(_df_ti['Approach']=='ssil') 
    & (_df_ti['Type']=='post_train')  & (_df_ti['Task']==1) & (_df_ti['Last App']==c)]['Time'].values[0]-0.2
    
    _df_ti.loc[_df_ti[(_df_ti['Approach']=='icarlp') & (_df_ti['Type']=='post_train')  &
    (_df_ti['Task']==1) & (_df_ti['Last App']==c)].index.values[0],'Time'] = _df_ti[(_df_ti['Approach']=='icarlp') 
    & (_df_ti['Type']=='post_train')  & (_df_ti['Task']==1) & (_df_ti['Last App']==c)]['Time'].values[0]-0.2

_df_ti.loc[_df_ti['Approach']=='joint', 'Approach'] = 'jointmem'
_df_ti.loc[_df_ti['Approach']=='backbonefreezing', 'Approach'] = 'backbonefreezingmem'
_df_ti.loc[_df_ti['Approach']=='lwf','Approach'] = 'lwfgkd'

_df_ti_39_mincpu = deepcopy(_df_ti)    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 670/670 [00:01<00:00, 438.31it/s]


In [280]:
#min gpu * epochs on gpu
#39+1 times
#load old experiments for info about epochs
exp_name = '39_1_base39_incr1_stop2'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_tr, df_ti = get_training_info(df_stdout_filenames)

#load new experiments (isolated)
exp_name= 'timing_base39'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr, _df_ti = get_training_info(df_stdout_filenames)
_df_tr = _df_tr[_df_tr['Seed']==0]
_df_tr = _df_tr[_df_tr['Last App']!=-1]
_df_ti = _df_ti[_df_ti['Last App']!=-1]

#load old scratch experiments for info about epochs
exp_name = 'mirage_generic_scratch_39_ub_base39_incr1_stop1'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/final_results/results_UB/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
_df_tr_scratch, _df_ti_scratch = get_training_info(df_stdout_filenames)

_df_tr, df_tr, _df_ti = clean_results(_df_tr, df_tr, _df_ti)

final_df = pd.concat((df_tr[df_tr['Task'] == 1], _df_tr_scratch[_df_tr_scratch['Task'] == 0]), axis=0)
final_df = final_df[final_df['Retrained App']==-1]

appr_list = ['joint', 'backbonefreezing' ]
t = df_ti[~((df_ti['Approach'].isin(appr_list)) & (df_ti['Memory Size'] == 0))]
df_ti = deepcopy(t)

final_gpu_time = pd.concat((df_ti[df_ti['Task'] == 1], _df_ti_scratch[_df_ti_scratch['Task'] == 0]), axis=0)
max_epochs = final_df.groupby(cols)['Epoch'].max().reset_index()

mean_time_per_epoch = df_tr[df_tr['Task'] == 1].groupby(cols)['Train Time'].min().reset_index()
mean_time_per_epoch_scratch = _df_tr_scratch[(_df_tr_scratch['Approach'] == 'scratch') & (_df_tr_scratch['Task'] == 1)].groupby(cols)['Train Time'].min().reset_index()

mean_time_per_epoch = pd.concat([mean_time_per_epoch_scratch, mean_time_per_epoch])

cols = ['Seed', 'Approach', 'Last App', 'First Momentum', 'Base Momentum', 'Network', 'Out Features Size']

columns = ['Approach', 'Seed', 'Last App']
groups = list(mean_time_per_epoch.groupby(columns).groups)
for group in tqdm(groups):
    _filter = reduce(lambda x, y: x & y, [mean_time_per_epoch[col] == g for col, g in zip(columns, group)])
    train_time = mean_time_per_epoch.loc[_filter]['Train Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, group)])
    if sum(_filter) == 0:
        continue
    max_epochs.loc[_filter, 'Train Time'] = train_time
    
max_epochs['Total Time'] = max_epochs[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)

groups = list(_df_ti.groupby(columns).groups)

failed_apprs = []
i = 0
for group in tqdm(groups):
    _group = group
    _filter = reduce(lambda x, y: x & y, [max_epochs[col] == g for col, g in zip(columns, _group)])
    if sum(_filter) == 0:
        failed_apprs.append(group[0])
        continue
    t = max_epochs.loc[_filter, 'Total Time'].values[0]
    _filter = reduce(lambda x, y: x & y, [final_gpu_time[col] == g for col, g in zip(columns, group)])
    if final_gpu_time.loc[(_filter)].Approach.unique()[0]!='scratch':
        final_gpu_time.loc[(_filter) & (final_gpu_time['Type'] == 'train') & (final_gpu_time['Task'] == 1), 'Time'] = t
    else:
        final_gpu_time.loc[(_filter) & (final_gpu_time['Type'] == 'train') & (final_gpu_time['Task'] == 0), 'Time'] = t 
    
#correzione chen2021 retraining time
max_epochs_chen = max_epochs[max_epochs['Approach'] == 'chen2021'][['Epoch', 'Train Time', 'Last App']]
df_tr_chen = df_tr[(df_tr['Approach'] == 'chen2021') & (df_tr['Task'] == 1) & (df_tr['Retrained App'] != -1)]
max_epochs_chen = max_epochs_chen[['Train Time', 'Last App']]
temp = df_tr_chen.groupby(['Last App', 'Retrained App'])['Epoch'].max().reset_index().groupby(['Last App']).sum().reset_index()
temp = pd.merge(temp, max_epochs_chen, on="Last App")
temp['Retraining Time'] = temp[['Epoch', 'Train Time']].apply(lambda x: x[0] * x[1], axis=1)
temp = pd.merge(temp[['Retraining Time', 'Last App']], _df_ti[(_df_ti['Approach']=='chen2021') & (_df_ti['Type']=='post_train') & (_df_ti['Task']==1)],on="Last App")
temp['Time'] = temp[['Retraining Time', 'Time']].apply(lambda x: x[0] + x[1], axis=1)

for c in temp['Last App'].unique():
    final_gpu_time.loc[final_gpu_time[(final_gpu_time['Approach']=='chen2021') & (final_gpu_time['Type']=='post_train') &
    (final_gpu_time['Task']==1) & (final_gpu_time['Last App']==c)].index.values[0],'Time'] = temp[temp['Last App']==c]['Time'].values[0]
    

final_gpu_time.loc[final_gpu_time['Approach']=='joint', 'Approach'] = 'jointmem'
final_gpu_time.loc[final_gpu_time['Approach']=='backbonefreezing', 'Approach'] = 'backbonefreezingmem'
final_gpu_time.loc[final_gpu_time['Approach']=='lwf','Approach'] = 'lwfgkd'

_df_ti_39_mingpu = deepcopy(final_gpu_time)    
_df_ti_39_mingpu.loc[_df_ti_39_mingpu.Approach=='scratch','Task'] = 1

#print(_df_ti_39_mingpu.groupby('Approach').describe())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 670/670 [00:01<00:00, 535.66it/s]


In [288]:
#39+1 plot times
types = [ 'all', 'post_train', 'pre_train', 'post_train_percentage', 'all_normalized_scratch', 'stacked'] 
for t in types:
    plt.close()
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_39, t, appr_task=1, group = 'Last App', nclasses=1, normalize=False, figsize=figsize)
    plt.savefig('ExecTime_Scenario_39+1_%s.pdf' % t, bbox_inches='tight')    
    
    
#39+1 plot times
types = [ 'all', 'post_train', 'pre_train', 'post_train_percentage', 'all_normalized_scratch', 'stacked'] 
for t in types:
    plt.close()
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_39_mincpu, t, appr_task=1, group = 'Last App', nclasses=1, normalize=False, figsize=figsize)
    plt.savefig('ExecTime_Scenario_39+1_%s_mincpu.pdf' % t, bbox_inches='tight') 

#39+1 plot times
types = [ 'all', 'post_train', 'pre_train', 'post_train_percentage', 'all_normalized_scratch', 'stacked'] 
for t in types:
    plt.close()
    figsize = (7, 4)
    fig, ax = plt.subplots(figsize=figsize)
    plot_train_time(_df_ti_39_mingpu, t, appr_task=1, group = 'Last App', nclasses=1, normalize=False, figsize=figsize)
    plt.savefig('ExecTime_Scenario_39+1_%s_mingpu.pdf' % t, bbox_inches='tight') 

   Approach     Pre_Train     Train    Post_Train
0       BiC  5.544439e-06  0.046926  3.012566e-03
1      EEIL  0.000000e+00  0.039611  1.192076e-02
2       EWC  0.000000e+00  0.042776  5.389897e-04
3    FT-Mem  0.000000e+00  0.027080  4.590553e-04
4    FZ-Mem  0.000000e+00  0.014615  4.615652e-04
5     iCaRL  0.000000e+00  0.054332  5.077120e-04
6    iCaRL+  0.000000e+00  0.047904  4.598943e-04
7      IL2M  1.343821e-08  0.029288  5.545313e-04
8     LUCIR  1.577101e-07  0.040530  4.575556e-04
9   LwF-GKD  0.000000e+00  0.023136  6.914722e-07
10  OvA-Ens  0.000000e+00  0.029005  1.987118e-02
11     SSIL  7.306143e-09  0.021222  4.531115e-04
   Approach     Pre_Train     Train    Post_Train
0       BiC  5.630484e-06  0.047164  3.058019e-03
1      EEIL  0.000000e+00  0.039837  1.210057e-02
2       EWC  0.000000e+00  0.042834  5.471422e-04
3    FT-Mem  0.000000e+00  0.027143  4.659693e-04
4    FZ-Mem  0.000000e+00  0.014674  4.684878e-04
5     iCaRL  0.000000e+00  0.054557  5.153812e-04


In [284]:
#load new experiments (controlled)
exp_name = 'timing_base20'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_training, df_timing = get_training_info(df_stdout_filenames)

#load new experiments (controlled) with 200 epochs
exp_name = 'timing_base20'
results_path = '/media/nas/datasets/MIRAGE_2020/FSCIL_approaches/hf-project/results/'
df_stdout_filenames = glob('%s/*%s*/timing200epochs/stdout*' % (results_path, exp_name), recursive=True)
df_stdout_filenames = discard_duplicates(df_stdout_filenames)
df_training200, df_timing200 = get_training_info(df_stdout_filenames)

_df_tr_eeil = df_training200[(df_training200.Approach=='eeil') & (df_training200.Task==1)]
_df_tr_eeil = _df_tr_eeil.drop(_df_tr_eeil.groupby(['Seed']).tail(40).index, axis=0)
df_training200_mod = df_training200[~(df_training200.Approach=='eeil')]
df_training200_mod = deepcopy(pd.concat([df_training200_mod, _df_tr_eeil]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 40.21it/s]


In [285]:
df_training200 = deepcopy(df_training200_mod)
approach_list = ['lucir', 'ewc', 'eeil', 'bic', 'lwf', 'chen2021']
print('----- Scratch -----')
seed = 1
max_epo = max_epochs[(max_epochs['Approach']=='scratch') & (max_epochs['Seed']==seed)]['Epoch'].values[0]
print('max_epo scratch:', max_epo)
min_all = df_training200[(df_training200['Approach']=='scratch') & (df_training200['Task']==1) & (df_training200['Seed']==seed)]['Train Time'].min()
scratch_all = min_all*max_epo
print('Time all epoches:', scratch_all, '\n------------------\n')
seeds = [1,2,3,4]

for approach in approach_list:
    ratios_3 = []
    ratios_all = []
    print ('Approccio:', approach, '\n')
    if approach=='chen2021': seeds = seeds[:-1]
    for seed in seeds:
        max_epo = max_epochs[(max_epochs['Approach']==approach) & (max_epochs['Seed']==seed)]['Epoch'].values[0]
        print ('Seed:', seed, ' - Num_epochs:', max_epo)
        min3 = df_training200[(df_training200['Approach']==approach) & (df_training200['Task']==1) & (df_training200['Seed']==seed)]['Train Time'].head(3).min()
        min_all = df_training200[(df_training200['Approach']==approach) & (df_training200['Task']==1) & (df_training200['Seed']==seed)]['Train Time'].min()
        print("Min 3 cpu epochs:", min3, " - Min all cpu epochs:", min_all)
        time_3 = min3*max_epo
        time_all = min_all*max_epo
        print('Time 3 epochs:', time_3, ' - Time all epochs:', time_all, '\n')
        ratios_3.append(time_3/scratch_all)
        ratios_all.append(time_all/scratch_all)
    print('Ratio min_3 epochs:', ratios_3)    
    print('Ratio min_all epochs:', ratios_all)
    sub = [a_i - b_i for a_i, b_i in zip(ratios_3, ratios_all)]
    print('Differences:', sub)
    print('Mean of differences: (across seeds):', np.mean(sub), '\n------------------\n')

----- Scratch -----
max_epo scratch: 173
Time all epoches: 3464.6710000000003 
------------------

Approccio: lucir 

Seed: 1  - Num_epochs: 200
Min 3 cpu epochs: 15.205  - Min all cpu epochs: 15.178
Time 3 epochs: 3041.0  - Time all epochs: 3035.6000000000004 

Seed: 2  - Num_epochs: 200
Min 3 cpu epochs: 14.1  - Min all cpu epochs: 13.975
Time 3 epochs: 2820.0  - Time all epochs: 2795.0 

Seed: 3  - Num_epochs: 179
Min 3 cpu epochs: 14.603  - Min all cpu epochs: 14.576
Time 3 epochs: 2613.937  - Time all epochs: 2609.1040000000003 

Seed: 4  - Num_epochs: 200
Min 3 cpu epochs: 17.349  - Min all cpu epochs: 17.205
Time 3 epochs: 3469.8  - Time all epochs: 3440.9999999999995 

Ratio min_3 epochs: [0.8777168164019036, 0.8139300961043631, 0.7544546076669328, 1.0014803714407514]
Ratio min_all epochs: [0.876158226856172, 0.8067144037630124, 0.7530596700235029, 0.9931678938635152]
Differences: [0.0015585895457316257, 0.007215692341350666, 0.0013949376434299054, 0.008312477577236188]
Mean of